# Feathr Feature Store on Azure Demo Notebook

This notebook illustrates the use of Feature Store to create a model that predicts NYC Taxi fares. It includes these steps:


This tutorial demonstrates the key capabilities of Feathr, including:

1. Install and set up Feathr with Azure
2. Create shareable features with Feathr feature definition configs.
3. Create a training dataset via point-in-time feature join.
4. Compute and write features.
5. Train a model using these features to predict fares.
6. Materialize feature value to online store.
7. Fetch feature value in real-time from online store for online scoring.

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The feature flow is as below:

![Feature Flow](https://github.com/linkedin/feathr/blob/main/docs/images/feature_flow.png?raw=true)

## Prerequisite: Provision cloud resources

First step is to provision required cloud resources if you want to use Feathr. Feathr provides a python based client to interact with cloud resources.

Please follow the steps [here]() to provision required cloud resources. Due to the complexity of the possible cloud environment, it is almost impossible to create a script that works for all the use cases. Because of this, [azure_resource_provision.sh](https://github.com/linkedin/feathr/blob/main/docs/how-to-guides/azure_resource_provision.sh) is a full end to end command line to create all the required resources, and you can tailor the script as needed, while [the companion documentation](https://github.com/linkedin/feathr/blob/main/docs/how-to-guides/azure-deployment.md) can be used as a complete guide for using that shell script.

At the end of the script, it should give you some output which you will need later. For example, the Service Principal IDs, Redis endpoint, etc.

Please also note that at the end of this step, you need to **manually** grant your service principal "Data Curator" permission of your Azure Purview account, due to a current limiation with Azure Purview.

And the architecture is as below:

![Architecture](https://github.com/linkedin/feathr/blob/main/docs/images/architecture.png?raw=true)

## Prerequisite: Install Feathr

Install Feathr using pip:

`pip install -U feathr pandavro scikit-learn`

Or if you want to use the latest Feathr code from GitHub:

`pip install -I git+https://github.com/linkedin/feathr.git#subdirectory=feathr_project pandavro scikit-learn`

## Prerequisite: Configure the required environment

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. If you use Feathr CLI to create a workspace, you should have a folder with a file called `feathr_config.yaml` in it with all the required configurations. Otherwise, update the configuration below.

The code below will write this configuration string to a temporary location and load it to Feathr. Please still refer to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It should also have more explanations on the meaning of each variable.

In [ ]:
import tempfile
yaml_config = """
# Please refer to https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml for explanations on the meaning of each field.
api_version: 1
project_config:
  project_name: 'udf_feathr_getting_started'
  required_environment_variables:
    - 'REDIS_PASSWORD'
    - 'AZURE_CLIENT_ID'
    - 'AZURE_TENANT_ID'
    - 'AZURE_CLIENT_SECRET'
offline_store:
  adls:
    adls_enabled: true
  wasb:
    wasb_enabled: true
  s3:
    s3_enabled: false
    s3_endpoint: 's3.amazonaws.com'
  jdbc:
    jdbc_enabled: false
    jdbc_database: 'feathrtestdb'
    jdbc_table: 'feathrtesttable'
  snowflake:
    url: "dqllago-ol19457.snowflakecomputing.com"
    user: "feathrintegration"
    role: "ACCOUNTADMIN"
spark_config:
  spark_cluster: 'azure_synapse'
  spark_result_output_parts: '1'
  azure_synapse:
    dev_url: 'https://feathrazuretest3synapse.dev.azuresynapse.net'
    pool_name: 'spark3'
    workspace_dir: 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started'
    executor_size: 'Small'
    executor_num: 4
    feathr_runtime_location: /Users/hnlin/IdeaProjects/feathr_release/target/scala-2.12/feathr-assembly-0.1.0.jar
  databricks:
    workspace_instance_url: 'https://adb-2474129336842816.16.azuredatabricks.net'
    config_template: {'run_name':'','new_cluster':{'spark_version':'9.1.x-scala2.12','node_type_id':'Standard_D3_v2','num_workers':2,'spark_conf':{}},'libraries':[{'jar':''}],'spark_jar_task':{'main_class_name':'','parameters':['']}}
    work_dir: 'dbfs:/feathr_getting_started'
    feathr_runtime_location: /Users/hnlin/IdeaProjects/feathr_release/target/scala-2.12/feathr-assembly-0.1.0.jar
online_store:
  redis:
    host: 'feathrazuretest3redis.redis.cache.windows.net'
    port: 6380
    ssl_enabled: True
feature_registry:
  purview:
    type_system_initialization: true
    purview_name: 'feathrazuretest3-purview1'
    delimiter: '__'
"""
tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)


## View the data

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The data is as below

In [ ]:
import pandas as pd
pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2020-04.csv")


In [ ]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


## Setup necessary environment variables

You should setup the environment variables in order to run this sample. More environment variables can be set by referring to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It also has more explanations on the meaning of each variable.

In [ ]:
os.environ['REDIS_PASSWORD'] = 'Li7Nn63iNB0x731VTnnz2Vr29WYJHx7JlAzCaH9lbHw='
os.environ['AZURE_CLIENT_ID'] = "b40e49c0-75c7-4959-ad25-896118cd79e8"
os.environ['AZURE_TENANT_ID'] = '72f988bf-86f1-41af-91ab-2d7cd011db47'
os.environ['AZURE_CLIENT_SECRET'] = 'kAB5ps6yvo_f08n-4Av~.IDwHFL_xl_63I'
os.environ['AZURE_PURVIEW_NAME'] = 'feathrazuretest3-purview1'


Then we will initialize a feathr client:


In [ ]:
client = FeathrClient(config_path=tmp.name)

## Defining Features with Feathr:

In Feathr, a feature is viewed as a function, mapping from entity id or key, and timestamp to a feature value. For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://github.com/linkedin/feathr/blob/main/docs/concepts/feature-definition.md)


1. The typed key (a.k.a. entity id) identifies the subject of feature, e.g. a user id, 123.
2. The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
3. The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.


Note that, in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.


### Define Sources Section
A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.


In [8]:
import os
from feathr.anchor import FeatureAnchor
from feathr.client import FeathrClient
from feathr.dtype import BOOLEAN, FLOAT, INT32, ValueType
from feathr.feature import Feature
from feathr.query_feature_list import FeatureQuery
from feathr.settings import ObservationSettings
from feathr.source import INPUT_CONTEXT, HdfsSource
from feathr.transformation import WindowAggTransformation
from feathr.typed_key import TypedKey
from pyspark.sql import SparkSession, DataFrame

def trip_distance_preprocessing(df: DataFrame):
    # give the user dataframe
    # user want to cast some fields to certain types
    print("This is my_func1: ")
    print(df)
    print(df.schema)
    # this is executed in spark
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    # df = df.filter("tolls_amount > 0.0")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    # df = df.withColumn("trip_distance", df.trip_distance.cast('double'))
    # df = df.withColumn("fare_amount", df.fare_amount.cast('string'))

    df = df.withColumn("trip_distance", df.trip_distance.cast('double') - 90000)
    df = df.withColumn("fare_amount", df.fare_amount.cast('double') - 90000)

    print(df)
    print(df.schema)
    df.show(10)
    return df

def my_func2(df: DataFrame):
    # give the user dataframe
    # user want to cast some fields to certain types
    print("This is my_func2: ")
    print(df)
    print(df.schema)
    # this is executed in spark
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")
    # df = df.filter("tolls_amount > 0.0")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    print("cast to double in my_func UDF!")
    # df = df.withColumn("trip_distance", df.trip_distance.cast('string'))
    # df = df.withColumn("fare_amount", df.fare_amount.cast('double'))

    df = df.withColumn("trip_distance", df.trip_distance.cast('double') + 90000)
    df = df.withColumn("fare_amount", df.fare_amount.cast('double') + 90000)
    print(df)
    print(df.schema)
    df.show(10)
    return df

# client = FeathrClient(config_path="/tmp/feathr_config.yaml")


batch_source = HdfsSource(name="nycTaxiBatchSource",
                          path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv",
                          event_timestamp_column="lpep_dropoff_datetime",
                          timestamp_format="yyyy-MM-dd HH:mm:ss")

location_id = TypedKey(key_column="lpep_pickup_datetime",
                       key_column_type=ValueType.INT32,
                       description="location id in NYC",
                       full_name="nyc_taxi.location_id")

# preprocessing
# figure out pyfile
# give the py func the input source data
# return the input source data back to Frame for further processing
f_fare_amount = Feature(name="f_fare_amount",
                          key=location_id,
                          feature_type=FLOAT, transform="fare_amount + 1000")
f_trip_time_duration = Feature(name="f_trip_time_duration",
                               key=location_id,
                               feature_type=INT32,
                               transform="time_duration(lpep_pickup_datetime, lpep_dropoff_datetime, 'minutes')")



features = [
    Feature(name="f_is_long_trip_distance",
            key=location_id,
            feature_type=FLOAT,
            transform="trip_distance + 1000"),
    Feature(name="f_day_of_week",
            key=location_id,
            feature_type=INT32,
            transform="dayofweek(lpep_dropoff_datetime)"),
]

request_anchor = FeatureAnchor(name="request_features",
                               source=batch_source,
                               features=features,
                               preprocessing=trip_distance_preprocessing)

features2222 = [
    f_fare_amount,
    f_trip_time_duration
]

feature_anchor2222 = FeatureAnchor(name="feature_anchor2222",
                               source=batch_source,
                               features=features2222,
                               preprocessing=my_func2)

# f_trip_time_distance = DerivedFeature(name="f_trip_time_distance",
#                                       feature_type=FLOAT,
#                                       input_features=[
#                                           f_trip_distance, f_trip_time_duration],
#                                       transform="f_trip_distance * f_trip_time_duration")
#
# f_trip_time_rounded = DerivedFeature(name="f_trip_time_rounded",
#                                      feature_type=INT32,
#                                      input_features=[f_trip_time_duration],
#                                      transform="f_trip_time_duration % 10")


agg_features = [Feature(name="f_location_avg_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        # transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                        transform=WindowAggTransformation(agg_expr="fare_amount",
                                                          agg_func="AVG",
                                                          window="90d")),
                Feature(name="f_location_max_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="MAX",
                                                          window="90d"))
                ]

agg_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=batch_source,
                           features=agg_features,
                           preprocessing=my_func2)

client.build_features(anchor_list=[agg_anchor, request_anchor, feature_anchor2222])
# client.build_features(anchor_list=[agg_anchor, request_anchor], derived_feature_list=[
#     f_trip_time_distance, f_trip_time_rounded])


# TODO
# anchor name and requested feature name might be different?
# {'f_location_avg_fare,f_location_max_fare': JavaObject id=o204, 'f_trip_distance,f_trip_time_duration,f_is_long_trip_distance,f_day_of_week': JavaObject id=o220}


feature_query = FeatureQuery(
    # feature_list=["f_is_long_trip_distance", "f_fare_amount", "f_trip_time_duration"], key=location_id)
    feature_list=["f_is_long_trip_distance", "f_day_of_week", "f_fare_amount", "f_trip_time_duration"], key=location_id)
    # feature_list=["f_location_avg_fare"], key=location_id)

settings = ObservationSettings(
    observation_path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv",
    event_timestamp_column="lpep_dropoff_datetime",
    timestamp_format="yyyy-MM-dd HH:mm:ss")

client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro",
                            udf_files=[]
                            )


2022-04-01 11:14:32.689 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:40 - Uploading /Users/hnlin/IdeaProjects/feathr_release/feathr_project/feathr/pyspark_client.py to cloud..
2022-04-01 11:14:32.690 | INFO     | feathr._synapse_submission:upload_file:335 - Uploading file pyspark_client.py


<function my_func2 at 0x1332c1ee0>
(['def my_func2(df: DataFrame):\n', '    # give the user dataframe\n', '    # user want to cast some fields to certain types\n', '    print("This is my_func2: ")\n', '    print(df)\n', '    print(df.schema)\n', '    # this is executed in spark\n', '    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")\n', '    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")\n', '    print("hi @feathr_udf!!!!!!!!!!!!!!!!!!!!!")\n', '    # df = df.filter("tolls_amount > 0.0")\n', '    print("cast to double in my_func UDF!")\n', '    print("cast to double in my_func UDF!")\n', '    print("cast to double in my_func UDF!")\n', '    print("cast to double in my_func UDF!")\n', '    # df = df.withColumn("trip_distance", df.trip_distance.cast(\'string\'))\n', '    # df = df.withColumn("fare_amount", df.fare_amount.cast(\'double\'))\n', '\n', '    df = df.withColumn("trip_distance", df.trip_distance.cast(\'double\') + 90000)\n', '    df = df.withColumn("fare_amount", df.fare_amount.cas

2022-04-01 11:14:33.098 | INFO     | feathr._synapse_submission:upload_file:341 - /Users/hnlin/IdeaProjects/feathr_release/feathr_project/feathr/pyspark_client.py is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/pyspark_client.py
2022-04-01 11:14:33.099 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:43 - /Users/hnlin/IdeaProjects/feathr_release/feathr_project/feathr/pyspark_client.py is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/pyspark_client.py
2022-04-01 11:14:33.100 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:40 - Uploading /var/folders/c2/fhs5jthx53q80vhsmkjhysq4000gby/T/tmp6_jqu9mo/client_udf_repo.py to cloud..
2022-04-01 11:14:33.101 | INFO     | feathr._synapse_submission:upload_file:335 - Uploading file client_udf_repo.py
2022-04-01 11:14:33.474 | INFO     | feathr._synapse_submission:upload_file

Argument: 
['--join-config', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf', '--input', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv', '--output', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro', '--feature-config', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/auto_generated_request_features.conf,abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/auto_generated_anchored_features.conf,abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/auto_generated_derived_features.conf', '--num-parts', '1', '--s3-config', '\n            S3_ENDPOINT: s3.amazonaws.com\n            S3_ACCESS_KEY: "None"\n            S3_SECRET_KEY: "None"\n            ', '--adls-config', '\n            ADLS_

2022-04-01 11:14:35.485 | INFO     | feathr._synapse_submission:upload_file:341 - /var/folders/c2/fhs5jthx53q80vhsmkjhysq4000gby/T/tmp6_jqu9mo/feature_join_conf/feature_join.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf
2022-04-01 11:14:35.486 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:43 - /var/folders/c2/fhs5jthx53q80vhsmkjhysq4000gby/T/tmp6_jqu9mo/feature_join_conf/feature_join.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf
2022-04-01 11:14:35.486 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:40 - Uploading /var/folders/c2/fhs5jthx53q80vhsmkjhysq4000gby/T/tmp6_jqu9mo/feature_conf/ to cloud..
2022-04-01 11:14:35.487 | INFO     | feathr._synapse_submission:upload_file_to_workdir:323 - Uploading folder /var/folders/c2/fhs5jthx53q80vhsmkjhysq4000gby/T/tmp6_

['abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/pyspark_client.py', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/client_udf_repo.py']
configuration: 
None
jars: 
[]
spark_batch_job_options: 
{'additional_properties': {}, 'tags': {'output_path': 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro'}, 'artifact_id': None, 'name': 'hangfei_release_feathr_getting_started_feathr_feature_join_job', 'file': 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/pyspark_client.py', 'class_name': 'com.linkedin.feathr.offline.job.FeatureJoinJob', 'arguments': ['--join-config', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf', '--input', 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv', '--o

## Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [ ]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    client.feathr_spark_laucher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(client)

In [ ]:
df_res

## Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [ ]:
# remove columns
from sklearn.ensemble import GradientBoostingRegressor
final_df = df_res
final_df.drop(["lpep_pickup_datetime", "lpep_dropoff_datetime",
              "store_and_fwd_flag"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['fare_amount'] = final_df['fare_amount'].astype("float64")


train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["fare_amount"], axis=1),
                                                    final_df["fare_amount"],
                                                    test_size=0.2,
                                                    random_state=42)
model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


## Materialize feature value into offline/online storage

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:


In [ ]:
backfill_time = BackfillTime(start=datetime(
    2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))
redisSink = RedisSink(table_name="nycTaxiDemoFeature")
settings = MaterializationSettings("nycTaxiTable",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["f_location_avg_fare", "f_location_max_fare"])

client.materialize_features(settings)
client.wait_job_to_finish(timeout_sec=500)


We can then get the features from the online store (Redis):


## Fetching feature value for online inference

For features that are already materialized by the previous step, their latest value can be queried via the client's
`get_online_features` or `multi_get_online_features` API.

In [ ]:
res = client.get_online_features('nycTaxiDemoFeature', '265', [
                                 'f_location_avg_fare', 'f_location_max_fare'])

In [ ]:
client.multi_get_online_features("nycTaxiDemoFeature", ["239", "265"], [
                                 'f_location_avg_fare', 'f_location_max_fare'])


### Registering and Fetching features

We can also register the features with an Apache Atlas compatible service, such as Azure Purview, and share the registered features across teams:

In [ ]:
client.register_features()
client.list_registered_features(project_name="feathr_getting_started")